<a href="https://colab.research.google.com/github/SushilKokare/AtiNeu_Project/blob/main/7_DenseNet121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121
from keras.layers import Dense,Flatten
from keras.models import Model

In [22]:
input_shape = (224,224)
batch_size=32

In [23]:
train_dir = '/content/drive/MyDrive/AtiNeu Project/Data/MRI/Brain_Data_Organised/Train'
test_dir = '/content/drive/MyDrive/AtiNeu Project/Data/MRI/Brain_Data_Organised/Test'

In [24]:
train_datagen = ImageDataGenerator (rescale = 1./225,)
test_datagen = ImageDataGenerator (rescale = 1./225)

In [25]:
train_gen  = train_datagen.flow_from_directory(
    train_dir,
    target_size = input_shape,
    class_mode = 'binary',
    batch_size = batch_size,
    classes=['Normal','Stroke']
)

Found 2251 images belonging to 2 classes.


In [26]:
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size = input_shape,
    batch_size = batch_size,
    class_mode = 'binary',
    classes=['Normal','Stroke']
)

Found 250 images belonging to 2 classes.


In [32]:
base_model = DenseNet121(weights='imagenet',include_top=False,
                         input_shape=(input_shape[0],input_shape[1],3))

29084464/29084464 [==============================] - 2s 0us/step


In [33]:
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D

In [35]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1,activation='sigmoid')(x)


In [37]:
model = Model(inputs=base_model.input,outputs=x)

In [38]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [61]:
history = model.fit(
    train_gen,
    epochs=100,
    validation_data=test_gen
)

Epoch 1/100
71/71 [==============================] - 26s 369ms/step - loss: 0.0505 - accuracy: 0.9822 - val_loss: 2.1886 - val_accuracy: 0.7160
Epoch 2/100
71/71 [==============================] - 26s 367ms/step - loss: 0.0179 - accuracy: 0.9956 - val_loss: 0.2487 - val_accuracy: 0.9080
Epoch 3/100
71/71 [==============================] - 26s 368ms/step - loss: 0.0885 - accuracy: 0.9725 - val_loss: 0.6145 - val_accuracy: 0.8240
Epoch 4/100
71/71 [==============================] - 26s 371ms/step - loss: 0.0438 - accuracy: 0.9853 - val_loss: 0.8108 - val_accuracy: 0.7800
Epoch 5/100
71/71 [==============================] - 27s 375ms/step - loss: 0.0094 - accuracy: 0.9987 - val_loss: 0.1934 - val_accuracy: 0.9320
Epoch 6/100
71/71 [==============================] - 28s 389ms/step - loss: 0.0058 - accuracy: 0.9991 - val_loss: 0.2530 - val_accuracy: 0.9240
Epoch 7/100
71/71 [==============================] - 27s 376ms/step - loss: 0.0688 - accuracy: 0.9756 - val_loss: 7.9168 - val_accuracy:

In [62]:
loss,acc = model.evaluate(test_gen)

8/8 [==============================] - 1s 133ms/step - loss: 1.4942 - accuracy: 0.8160


In [63]:
print('Acc ',acc)

Acc  0.8159999847412109


In [64]:
import pandas as pd

In [65]:
pd.DataFrame(history.history)

,loss,accuracy,val_loss,val_accuracy
0,0.050504,0.982230,2.188615,0.716
1,0.017908,0.995558,0.248708,0.908
2,0.088453,0.972457,0.614519,0.824
3,0.043804,0.985340,0.810811,0.780
4,0.009433,0.998667,0.193366,0.932
...,...,...,...,...
95,0.000157,1.000000,0.173982,0.952
96,0.169459,0.943136,6.422485,0.544
97,0.039857,0.986673,0.322117,0.928
98,0.008343,0.997779,0.266385,0.912


In [66]:
import matplotlib.pyplot as plt

In [68]:
# plt.figure()
# plt.xlabel('Epochs')
# plt.ylabel('Acc')
# for i in ['accuracy','val_accuracy']:
#   x = history.history[i]
#   plt.plot(x,label=i)
# plt.legend()